# Single instance explanations

In [1]:
import pandas as pd
import numpy as np
from SVM_model import SVM_model
from Functions import separate_bins_feature

# Assuming a model that gives % of 'goodness' for any sample

# Assuming that for categorical features there will be only values in range 0-7. 

# Hardcodes the constraints for the direction in which to move
# 1: Move up to to improve
# 0: Move down to improve
# -1: Needs check
monotonicity_arr = [1,1,1,1,1,0,0,1,1,1,1,-1,0,-1,1,0,0,0,0,-1,-1,0,-1]

In [2]:
# Create arrays with mean values for each bin and the position in the bin of each value

# X = original data set (9871x23)
df = pd.read_csv("xML Challenge Dataset and Data Dictionary/working_data_full.csv")
vals = df.values
X = vals[:,2:]
y_original= vals[:,1]

samples = X.shape[0]
features = X.shape[1]
mean_values_bins = []
X_bined_positions = []

for i in range (features):
    if i in [9,10]:
        result_bins = np.array([0,1,2,3,4,5,6,7,-1,-1])
        result_new_col = X[:,i].flatten()
    else:
        result_bins, result_new_col = separate_bins_feature(X[:,i].flatten())[:2]
    mean_values_bins.append(result_bins)
    X_bined_positions.append(result_new_col)
    
mean_values_bins = np.array(mean_values_bins)
X_bined_positions = np.array(X_bined_positions)  
X_bined_positions = np.transpose(X_bined_positions)

print(mean_values_bins, mean_values_bins.shape)
print(X_bined_positions, X_bined_positions.shape)

[[  54.   58.   62.   66.   70.   74.   78.   82.   86.   90.]
 [  25.   63.  101.  139.  177.  215.  253.  291.  329.  367.]
 [   1.    4.    7.   10.   13.   16.   19.   22.   25.   28.]
 [  17.   30.   43.   56.   69.   82.   95.  108.  121.  134.]
 [   2.    6.   10.   14.   18.   22.   26.   30.   34.   38.]
 [   0.    1.    2.    3.   -1.   -1.   -1.   -1.   -1.   -1.]
 [   0.    1.    2.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [  71.   75.   79.   83.   87.   91.   95.   99.  103.  107.]
 [  10.   31.   52.   73.   94.  115.  136.  157.  178.  199.]
 [   0.    1.    2.    3.    4.    5.    6.    7.   -1.   -1.]
 [   0.    1.    2.    3.    4.    5.    6.    7.   -1.   -1.]
 [   2.    6.   10.   14.   18.   22.   26.   30.   34.   38.]
 [   0.    1.    2.    3.    4.    5.    6.   -1.   -1.   -1.]
 [   3.   10.   17.   24.   31.   38.   45.   52.   59.   66.]
 [   7.   21.   35.   49.   63.   77.   91.  105.  119.  133.]
 [   0.    1.    2.    3.    4.    5.    6.   -1.   -1.

In [3]:
svm_model = SVM_model(None, "xML Challenge Dataset and Data Dictionary/working_data_full.csv")

svm_model.train_model(0.01)
svm_model.test_model()

Training Accuracy: 72.64 %
Test Accuracy: 75.04 %


In [16]:
def perturb_row_feature(model, row, row_idx, feat_idx, current_bins):
    
    c_current_bins = np.copy(current_bins)
    direction = monotonicity_arr[feat_idx]
    current_bin = np.copy(c_current_bins[feat_idx])
    
    if current_bin != 9:
        next_value = mean_values_bins[feat_idx][int(current_bin+1)]
    if current_bin != 0:
        prev_value = mean_values_bins[feat_idx][int(X_bined_positions[row_idx][feat_idx]-1)]
    
    # Check if in boundary and return the same row
    if direction == -1:
        if current_bin == 0:
            direction = 1
        elif current_bin == 9 or next_value == -1:
            direction = 0
    if direction == 1:
        if current_bin == 9 or next_value == -1:
            return (row, c_current_bins)
    elif direction == 0 and current_bin ==  0:
            return (row, c_current_bins)
    
    # Decide direction in special case
    if direction == -1:
        row_up = np.copy(row)
        row_down = np.copy(row)
        row_up[feat_idx] = next_value
        row_down[feat_idx] = prev_value
        percent_1 = model.run_model(row_up)
        percent_0 = model.run_model(row_down)
        if percent_1 >= percent_0:
            c_current_bins[feat_idx] += 1
            return (row_up, c_current_bins)
        else:
            c_current_bins[feat_idx] -= 1
            return (row_down, c_current_bins)
        
    else:
        p_row = np.copy(row)
        if direction == 1:
            c_current_bins[feat_idx] += 1
            p_row[feat_idx] = next_value
        elif direction == 0:
            c_current_bins[feat_idx] -= 1
            p_row[feat_idx] = prev_value
        
        return (p_row, c_current_bins)

def instance_explanation(model, k_row, row_idx):
    
    row = np.copy(k_row)
    percent = model.run_model(row)
    features_moved = np.zeros(23)
    times_moved = np.zeros(23)
    change_vector = np.zeros(23)
    
    original_bins = np.copy(X_bined_positions[row_idx])
    current_bins = np.copy(X_bined_positions[row_idx])
       
    while percent <= 0.5 and (features_moved == 1).sum() <= 5 and max(times_moved) < 5:
    
        new_percents = []
        pert_rows = []
        new_curr_bins = []
        
        for i in range(len(row)):
            t_row, t_current_bins = perturb_row_feature(model, row, row_idx, i, current_bins)
            pert_rows.append(t_row)
            new_curr_bins.append(t_current_bins)
            new_percents.append(model.run_model(t_row))

        new_percents = np.array(new_percents)
        idx = np.argmax(new_percents)
        
        row = pert_rows[idx]
        percent = new_percents[idx]
        current_bins = new_curr_bins[idx]

        features_moved[idx] = 1
        times_moved[idx] += 1
    
    for l in range(23):
        change_vector[l] = current_bins[l] - original_bins[l]
        
    if percent >= 0.5:
        return change_vector, row
    else:
        return ("Decision can't be moved within thresholds.")

In [17]:
for i in range(samples):
    print(i, instance_explanation(svm_model, X[i], i)[0])

0 D
1 D
2 D
3 D
4 [ 0.  0.  0.  1.  3.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0. -1.
  0.  0.  0. -1.  0.]
5 D
6 [ 0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0. -1.  0.]
7 [ 0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.
  0.  0.  0. -1.  0.]
8 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
9 D
10 [ 0.  0.  0.  4.  4.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0. -1.
  0.  0.  0. -1.  0.]
11 D
12 D
13 D
14 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
15 D
16 D
17 [ 0.  0.  0.  1.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
18 D
19 D
20 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
21 D
22 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
23 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

144 D
145 D
146 [ 0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. -1.  0.  0.  0.]
147 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
148 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
149 D
150 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
151 D
152 D
153 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
154 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
155 D
156 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
157 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
158 [ 0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
159 [ 0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

267 [ 0.  0.  0.  5.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.
  0.  0.  0.  0.  0.]
268 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.
  0.  0.  0. -1.  0.]
269 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
270 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
271 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
272 D
273 D
274 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
275 D
276 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
277 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
278 D
279 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
280 [ 0.  0.  0.  4.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.
  0.  0.

397 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
398 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
399 [ 0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.  0. -1.  0.]
400 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
401 D
402 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
403 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
404 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.
  0.  0.  0. -1.  0.]
405 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
406 D
407 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
408 [ 0.  0.  0.  5.  3.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.
  0.  0.  0. -1.  0.

539 D
540 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
541 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
542 D
543 D
544 D
545 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
546 D
547 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
548 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.
  0.  0.  0.  0.  0.]
549 D
550 D
551 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. -1.  0.  0.  0.]
552 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
553 [ 0.  0.  0.  1.  4.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.
  0.  0.  0. -1.  0.]
554 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
555 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

...

1222 D
1223 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1224 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1225 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1226 D
1227 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1228 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1229 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1230 D
1231 [ 0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.
  0.  0.  0. -1.  0.]
1232 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. -1.  0. -1.  0.]
1233 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.
  0.  0.  0. -1.  0.]
1234 D
1235 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

1338 D
1339 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1340 D
1341 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1342 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1343 D
1344 D
1345 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1346 [ 0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1347 D
1348 D
1349 D
1350 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1351 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1352 D
1353 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1354 D
1355 D
1356 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1357 [ 0.  0.  0.  0.  0.  0. 

1468 D
1469 D
1470 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1471 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1472 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1473 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1474 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1475 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1476 [ 0.  0.  0.  1.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.
  0.  0.  0.  0.  0.]
1477 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1478 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1479 D
1480 [ 0.  0.  0.  2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
 

1600 [ 0.  0.  0.  1.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.
  0.  0.  0.  0.  0.]
1601 D
1602 D
1603 D
1604 D
1605 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. -1.  0. -1.  0.]
1606 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1607 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1608 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1609 D
1610 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1611 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1612 [ 0.  0.  0.  0.  4.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1613 [ 0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]
1614 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

KeyboardInterrupt: 